In [2]:
import neuroglancer
import cloudvolume
import pandas as pd, os

## README
This example assumes you have precomputed datasets already made and hosted with cloudvolume.
To host the raw data in this example that I already converted to precomputed format, open an ipython terminal and do:
```python
import cloudvolume 
vol = cloudvolume.CloudVolume('file:///jukebox/scratch/zmd/contra_ipsi_projection_studies_20191125/20180322_jg_bl6f_prv_28/647') 
vol.viewer(port=1344) # You can choose the port here, but remember it for later
```

```python
import cloudvolume 
vol = cloudvolume.CloudVolume('file:///jukebox/scratch/zmd/contra_ipsi_projection_studies_20191125/20180322_jg_bl6f_prv_28/atlas') 
vol.viewer(port=1345) # You can choose the port here, but remember it for later
```

The last command will cause the window to hang -- that is the expected behavior. Do not try to do this in this jupyter notebook as it will cause your session to hang and you won't be able to run any of the following steps.

## Set the neuroglancer client you want to use. 
Default is Seung lab's which should work fine for this example

In [3]:
neuroglancer.set_static_content_source(url='https://neuromancer-seung-import.appspot.com')

# Load in the raw data into neuroglancer and generate the link

In [4]:
# This volume handle can be used to notify the viewer that the data has changed.
viewer = neuroglancer.Viewer()

# Load in a segmentation layer (e.g. the raw-space allen atlas) that is being hosted with cloudvolume
with viewer.txn() as s:
    s.layers['20180322_jg_bl6f_prv_28'] = neuroglancer.ImageLayer(source='precomputed://http://localhost:1344',
    shader = '''
    void main() {
  float v = toNormalized(getDataValue(0)) * 30.0;
  float w = 1.0 - v;
  emitRGBA(vec4(w, w, w, v));
}
'''                                                 
    )    
    s.layers['rawatlas_20180322_jg_bl6f_prv_28'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1345'
    )

print(viewer)

http://127.0.0.1:34561/v/0b200c25b1af51d71bfae8b11e54f258649fdb15/


Use the look-up table to make a key binding to the Neuroglancer session.

In [5]:
# First you need to create a dictionary mapping the atlas id to the region name. 
# Here's my example

csv_file = '/jukebox/LightSheetTransfer/atlas/ls_id_table_w_voxelcounts_16bit.xlsx'

df = pd.read_excel(csv_file)

atlas_df = df.copy()

atlas_dict = {}
for ind, row in atlas_df.iterrows():
    atlas_dict[int(row['id'])] = row['name']
    
# Here is the actual code for making the key binding. Copy and paste this
# into your jupyter notebook where you have already made the viewer() object

num_actions = 0
def my_action3(s):
    global num_actions
    num_actions += 1
    with viewer.config_state.txn() as st:
        region_id = s.selected_values['rawatlas_20180322_jg_bl6f_prv_28']
        region_name = atlas_dict.get(region_id)
        st.status_messages['hello'] = ('%i:%s' %
                                    (region_id, region_name))

    print('Got my-action %i' % num_actions)
#     print('  Mouse position: %s' % (s.mouse_voxel_coordinates,))
    print('  Layer selected values: %s' % (s.selected_values,))
viewer.actions.add('my-action3', my_action3)

with viewer.config_state.txn() as s:
    s.input_event_bindings.viewer['keyp'] = 'my-action3'
    s.status_messages['hello'] = 'Welcome to the segment labeling example. Press "p" to see region name under cursor.'

## Take screenshots (optional)

Adjust the viewer and take a screenshot. Good for videos!

In [18]:
svdst = '/jukebox/wang/zahra/neuroglancer/screenshots/20170115_tp_bl6_lob6b_ml_04/frontal_areas'
#make sure these directories exist
if not os.path.exists(os.path.dirname(svdst)): os.mkdir(os.path.dirname(svdst)) #brain directory
if not os.path.exists(svdst): os.mkdir(svdst) #structure directory
for i in range(180, 530):
    with viewer.config_state.txn() as s:
        s.show_ui_controls = False
        s.show_panel_borders = False
    with viewer.txn() as s:
        s.voxel_coordinates = [3081, 1314,i]
#    with viewer.config_state.txn() as s:
#        s.viewer_size = [1000,1000]
    ss = neuroglancer.ScreenshotSaver(viewer, svdst)
    ss.capture(index=i)   

Alternatively, can also take a single view screenshot of the current view.

In [10]:
svdst = '/jukebox/wang/zahra/neuroglancer/screenshots/20170115_tp_bl6_lob6b_ml_04/frontal_areas'
if not os.path.exists(svdst): os.mkdir(svdst)
with viewer.config_state.txn() as s:
    s.show_ui_controls = False
    s.show_panel_borders = False
    s.viewer_size = [1000, 1000]
ss = neuroglancer.ScreenshotSaver(viewer, svdst)
ss.capture(index=0) 

(0,
 '/jukebox/wang/zahra/neuroglancer/screenshots/20170115_tp_bl6_lob6b_ml_04/frontal_areas/0000000.png')

Return viewer state to normal:

In [23]:
with viewer.config_state.txn() as s:
    s.show_ui_controls = True
    s.show_panel_borders = True
    s.viewer_size = None

Change the view layout to show the segmentation side by side with the image, rather than overlayed.  This can also be done from the UI by dragging and dropping.  The side by side views by default have synchronized position, orientation, and zoom level, but this can be changed.

In [69]:
with viewer.txn() as s:
    s.layout = neuroglancer.row_layout(
        [neuroglancer.LayerGroupViewer(layers=['20160822_tp_bl6_crii_1500r_06', 'overlay']),
         neuroglancer.LayerGroupViewer(layers=['rawatlas_20160822_tp_bl6_crii_1500r_06'])])

Remove the overlay layer.

In [6]:
with viewer.txn() as s:
    s.layout = neuroglancer.row_layout(
        [neuroglancer.LayerGroupViewer(layers=['20160822_tp_bl6_crii_1500r_06']),
         neuroglancer.LayerGroupViewer(layers=['20170207_db_bl6_crii_1300r_02']),
        #neuroglancer.LayerGroupViewer(layers=['20161205_tp_bl6_lob45_1000r_01']),
        neuroglancer.LayerGroupViewer(layers=['20170116_tp_bl6_lob6b_lpv_07']),
        neuroglancer.LayerGroupViewer(layers=['20170411_db_bl6_crii_mid_53hr']),
        neuroglancer.LayerGroupViewer(layers=['20170419_db_bl6_cri_rpv_53hr'])])